In [ ]:
import tensorflow as tf
from keras import models, layers

# Path to dataset (must contain 'cats' and 'dogs' subfolders)
dataset_path = "../data/PetImages/"

# Load full dataset
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42
)

In [ ]:
# Take a smaller subset (e.g., 1% of the data)
fraction = 0.02
small_train_size = int(fraction * len(full_dataset))  # 10% for training
small_val_size = int(fraction * len(full_dataset))  # 10% for validation

small_train_dataset = full_dataset.take(small_train_size)
small_val_dataset = full_dataset.skip(small_train_size).take(small_val_size)

# Normalize and optimize
AUTOTUNE = tf.data.AUTOTUNE
small_train_dataset = small_train_dataset.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))
small_train_dataset = small_train_dataset.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)

small_val_dataset = small_val_dataset.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))
small_val_dataset = small_val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 24998 files belonging to 2 classes.


In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))),
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

/home/pewhite/github/aet-cs/aet-cs.github.io/white/ML/tf-env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 173056)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    11,075,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,132,033 (42.47 MB)

 Trainable params: 11,132,033 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    small_train_dataset,
    validation_data=small_val_dataset,
    epochs=10  # Adjust as needed
)


Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.5488 - loss: 213.2742  

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


78/78 ━━━━━━━━━━━━━━━━━━━━ 41s 510ms/step - accuracy: 0.5489 - loss: 211.2926 - val_accuracy: 0.5385 - val_loss: 0.6880
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 43s 548ms/step - accuracy: 0.6892 - loss: 0.5824 - val_accuracy: 0.5541 - val_loss: 0.7422
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 50s 643ms/step - accuracy: 0.7967 - loss: 0.4127 - val_accuracy: 0.5489 - val_loss: 0.8710
Epoch 4/10
17/78 ━━━━━━━━━━━━━━━━━━━━ 28s 470ms/step - accuracy: 0.9179 - loss: 0.2535

KeyboardInterrupt: 

In [28]:
from tensorflow.keras.preprocessing import image
import os

dataset_path = "../data/PetImages/"

def check_image(path):
    try:
        img = image.load_img(path)  # Try loading the image
        img_array = image.img_to_array(img)  # Try converting it to an array
        return True  # Image is valid
    except Exception as e:
        print(f"Corrupt image: {path} | Error: {e}")
        return False

# Check all images in the dataset
for subfolder in ['Cat', 'Dog']:
    folder_path = os.path.join(dataset_path, subfolder)
    for img_filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_filename)
        if not check_image(img_path):
            print(f"Removing corrupt image: {img_path}")
            os.remove(img_path)


Corrupt image: ../data/PetImages/Cat/Thumbs.db | Error: cannot identify image file <_io.BytesIO object at 0x79769b3dfa60>
Removing corrupt image: ../data/PetImages/Cat/Thumbs.db
Corrupt image: ../data/PetImages/Cat/666.jpg | Error: cannot identify image file <_io.BytesIO object at 0x79769b789530>
Removing corrupt image: ../data/PetImages/Cat/666.jpg


/home/pewhite/github/aet-cs/aet-cs.github.io/white/ML/tf-env/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Corrupt image: ../data/PetImages/Dog/11702.jpg | Error: cannot identify image file <_io.BytesIO object at 0x79769a2dfc90>
Removing corrupt image: ../data/PetImages/Dog/11702.jpg
Corrupt image: ../data/PetImages/Dog/Thumbs.db | Error: cannot identify image file <_io.BytesIO object at 0x797699ce8ef0>
Removing corrupt image: ../data/PetImages/Dog/Thumbs.db
